<a href="https://colab.research.google.com/github/newfrogg/Automatic-Differentiation/blob/main/autodiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Mục lớn                             | Mục nhỏ                                                         |           |
|-------------------------------------|-----------------------------------------------------------------|-----------|
| Introduction                        |                                                                 |  1 người  |
|                                     | Differential Calculus                                           |  Tử Quân  |
|                                     | Rules of Calculus                                               |  Tử Quân  |
|                                     | Multivariate Chain Rule                                         |  Tử Quân  |
|                                     | Geometry of Gradients and Gradient Descent                      |  Tử Quân  |
| What Autodiff Isn’t                 |                                                                 |  3 người  |
|                                     | Autodiff is not finite differences (numerical differentiation)  |  An Đông  |
|                                     | Autodiff is not symbolic differentiation                        |  An Đông  |
|                                     | What autodiff is ?                                              |  An Đông  |
|                                     | Types of Autodiff (explain forward, backward)                   |  An Đông  |
|                                     | Backpropagation Algorithm                                       |  Gia Hinh |
|                                     | Gradient-Based Optimization                                     |  Gia Hinh |
|                                     | give an example and describe the autodiff step-by-step workflow |  Gia Hinh |
|                                     | Summary. Compare betwwen each type advancement of each types    |  Gia Hinh |
| Visualization. Code, Implementation |                                                                 |  1 người  |
|                                     | Code                                                            | Bảo Lương |
| Exercise                            |                                                                 |  2 người  |
|                                     | Q1-4                                                            |   Triết   |
|                                     | Q5-8                                                            | Minh Quân |

# Introduction

## Differential Calculus
## Rules of Calculus
## Geometry of Gradients and Gradient Descent

## Multivariate Chain Rule
## The Backpropagation Algorithm

https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/single-variable-calculus.html
https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/multivariable-calculus.html

<div align="justify">

## **What AD Is Not ?**
Without a clear introduction, many people place automatic differentiation (AD) in the same category as numerical differentiation or symbolic differentiation. This confusion is understandable because AD delivers numerical values for derivatives while using the rules of symbolic calculus, yet it only stores the resulting numbers rather than the complete algebraic expressions. This dual character means AD operates in a space that overlaps both approaches (Griewank, 2003). To start, we will explain how AD differs from those two familiar methods and in many respects exceeds their capabilities.

### **AD Is Not Numerical Differentiation**
Numerical differentiation estimates a derivative by applying finite difference formulas to function values sampled at specific points (Burden & Faires, 2001). At its core, the method springs directly from the limit definition of the derivative. For example, for a multivariate function $f∶R^n→R$, one can approximate the gradient $(\frac{∂f}{∂x_1 },…,\frac{∂f}{∂x_n})$ using:

$$\frac{∂f(x)}{∂x_i} \approx \frac{f(x+he_i )-f(x)}{h}\text{ (Eq.1)}$$


where $e_i$ is the $i$-th unit vector and $h>0$ is a small step size. Its main upside is simplicity of implementation; the downsides are that computing a gradient in $n$ dimensions needs $O(n)$ separate evaluations of $f$ and that you must choose the step size $h$ with great care.

Approximating derivatives numerically is fundamentally unstable and suffers from poor conditioning, except when using complex variable techniques that apply only to certain holomorphic functions (Fornberg, 1981). This instability arises from truncation errors and round off errors introduced by finite computational precision and the selection of the step size $h$. While the truncation error vanishes as $h$ approaches zero $(h→0)$, reducing $h$ increases the round off error until it becomes the primary source of inaccuracy.

Various methods have been proposed to reduce the approximation error in numerical differentiation. For example, using a central difference formula:

$$ \frac{∂f(x)}{∂x_i} \approx \frac{f(x+he_i )-f(x-he_i)}{2h} + O(h^2 )\text{ (Eq.2)}$$

in which the leading order errors cancel out, pushing the truncation error from first order to second order in $h$. In one dimension, this central difference (Eq. 2) is just as expensive as the forward difference (Eq. 1) because both require two evaluations of $f$. But in higher dimensions the cost grows quickly: forming the full Jacobian of a map $f∶R^n→R^m$ using central differences demands $2mn$ function evaluations, forcing a trade off between accuracy and computational expense.

Advanced approaches to enhance numerical differentiation, such as higher order finite difference schemes, Richardson extrapolation toward the limit (Brezinski & Zaglia, 1991), and weighted sum differential quadrature methods (Bert & Malik, 1996), all raise computational cost, never fully remove approximation errors, and remain highly vulnerable to floating point truncation. Moreover, the $O(n)$ expense of computing an $n$-dimensional gradient makes numerical differentiation impractical for machine learning, where $n$ can reach millions or billions in cutting edge deep networks (Shazeer et al., 2017). By contrast, deep learning models tolerate approximation errors reasonably well thanks to their intrinsic resilience to numerical noise (Gupta et al., 2015).

### **AD Is Not Symbolic Differentiation**

Symbolic differentiation refers to the automated transformation of mathematical expressions to derive their exact derivative formulas (Grabmeier and Kaltofen, 2003). This is achieved by systematically applying differentiation rules, for example:

$$ \frac{d}{dx}(f(x)+g(x)) \approx \frac{d}{dx}(f(x)) + \frac{d}{dx}(g(x))\text{ (Eq.3)}$$

$$ \frac{d}{dx}(f(x)g(x)) \approx (\frac{d}{dx}(f(x)))g(x) + f(x)(\frac{d}{dx}(g(x)))\text{ (Eq.4)}$$

When mathematical formulas are stored as structured data, differentiating an expression tree by symbolic means becomes a fully mechanical task, an idea that dates back to the earliest days of calculus (Leibniz, 1685). Today, this approach is implemented in computer algebra systems like Mathematica, Maxima, and Maple, as well as in machine learning libraries such as Theano.

In optimization, symbolic derivatives can reveal the underlying structure of a problem and sometimes even yield closed-form solutions for extrema( such as solving for $\frac{d}{dx} f(x)=0$) bypassing the need for further derivative evaluations. However, symbolic derivatives often expand into expressions that grow exponentially in size compared to the original formula, making them impractical for efficient runtime computation of derivative values.

Take the function $h(x)=f(x)h(x)$ and apply the product rule. Both $h(x)$ and its derivative $(\frac{d}{dx} h(x))$ share the subexpressions $f(x)$ and $g(x)$. On the derivative’s right hand side, the terms $f(x)$ and $\frac{d}{dx} f(x)$ appear separately. If you immediately expand $\frac{d}{dx} f(x)$ by plugging in its symbolic derivative, you duplicate every computation common to $f(x)$ and $\frac{d}{dx} f(x)$. Repeating this process without optimization causes the symbolic representation to grow exponentially, making evaluation extremely slow. This issue is called expression swell.

When our focus is on obtaining accurate numerical derivatives rather than preserving their symbolic representations, we can greatly reduce computational effort by storing only the numerical values of intermediate sub expressions. To make this even more efficient, we interleave differentiation and simplification at each step. This concept is the foundation of automatic differentiation: at every elementary operation, perform the symbolic differentiation while simultaneously tracking and storing the numerical results alongside the primary function’s computation. This procedure defines forward mode AD, which we will discuss in the next section.

## **What is Autodiff and type of Autodiff ?**
### **What is Autodiff ?**

Automatic differentiation can be viewed as an alternative way to execute a program in which the original calculations are extended to include derivative computations. Every numerical routine ultimately breaks down into a finite collection of basic operations whose derivatives are known (Verma, 2000; Griewank and Walther, 2008). By applying the chain rule to combine the derivatives of these basic operations, one obtains the derivative of the entire composite function. Common basic operations include addition, subtraction, multiplication, division, the sign operation, and transcendental functions such as exponentials, logarithms, and trigonometric functions.

To implement this, we can leverage an evaluation trace. An evaluation trace is a special table that keeps track of intermediate variables as well as the operations that created them. Every row corresponds to an intermediate variable and the elementary operation that caused it. These variables, called primals, are typically denoted $v_i$ for functions $f:R^n→R^m$ and follow these rules:

- Input variables: $v_{i-n}=x_i,i=1,…,n$
- Intermediate variables: $v_i,i=1,…,l$
- Output variables: $y_{m-i}=v_{l-i}  ,i=m-1,…,0$



Example:

$$y=f(x_1,x_2 )=x_1 x_2+x_2-ln⁡(x_1 )\text{ (Eq.5)}$$
$$x_1=2,x_2=4$$

<div align="center">

| **Forward Primal Trace** | **Output** |
| -------- | ------- |
| $v_{-1}=x_1$ | 2 |
| $v_{0}=x_2$ | 4 |
| $v_1=v_{-1} \times v_0$ | 2 x (4) = 8 |
| $v_2=ln⁡(v_{-1})$ | ln(2) = 0.693 |
| $v_3=v_1+v_0$ | 8 + 4 = 12 |
| $v_4=v_2-v_3$ | 12 – 0.693 = 11.307 |
| $y=v_4$ | 11.307 |
<p style="text-align:center;">Table 1: Forward Primal Trace</p>
</div>

![image](./image/compution_graph.png "Computation Graph")
<p style="text-align:center;">Figure 1: Computation Graph</p>

Evaluation traces are fundamental to automatic differentiation. Unlike symbolic differentiation, which is restricted to closed form expressions, AD can also handle algorithms that include branching, loops, recursion, and function calls. This flexibility stems from the fact that any numerical program execution generates a trace of concrete input, intermediate, and output values. Those values alone suffice to calculate derivatives via repeated application of the chain rule, regardless of which control flow path the code followed. In other words, AD disregards any operations, such as conditionals or loop controls, that do not directly affect numerical values.

### **Forward Mode**

Forward mode AD enhances the evaluation trace by linking each primary value $v_i$ to a tangent $\dot{v_i}$. These tangents record the partial derivative of each value relative to the selected input variable.

Referencing back to eq. 5, we'd have the following definition of tangents if we were interested in finding $\frac{∂y}{∂x_2 }$:

$$ \dot{v_i} = \frac{∂v_i}{∂x_2} $$

Continuing from this definition, we can build out the forward primal and forward tangent trace to compute $\frac{∂y}{∂x_2}$ when $x_1=3,x_2=-4,\dot{x_1}=\frac{∂x_1}{∂x_2}=0$, and $\dot{x_2}=\frac{∂x_2}{∂x_2}=1$.

<div align="center">

| Forward Primal Trace | Output | Forward Tangent Trace                       | Output           |
|----------------------|--------|---------------------------------------------|------------------|
| $v_{-1} = x_1$       | 3      | $\dot v_{-1} = \dot x_1$                    | 0                |
| $v_{0} = x_2$        | –4     | $\dot v_{0} = \dot x_2$                     | 1                |
| $v_{1} = v_{-1} \times v_{0}$ | $3 \times (-4) = -12$ | $\dot v_{1} = \dot v_{-1}\,v_{0} + \dot v_{0}\,v_{-1}$ | $0\times(-4) + 1\times(3) = 3$ |
| $v_{2} = \ln(v_{-1})$| $\ln(3)=1.10$ | $\dot v_{2} = \dot v_{-1}\times\frac{1}{v_{-1}}$ | $0\times\frac1{3}=0$ |
| $v_{3} = v_{1} + v_{0}$ | $-12 + -4 = -16$ | $\dot v_{3} = \dot v_{1} + \dot v_{0}$ | $3 + 1 = 4$ |
| $v_{4} = v_{2} - v_{3}$ | $-16 - 1.10 = 17.10$ | $\dot v_{4} = \dot v_{2} - \dot v_{3}$ | $0 - 4 = -4$ |
| $y = v_{4}$ | $-17.10$ | $\dot y = \dot v_{4}$ | $-4$ |

<p style="text-align:center;">Table 2: Forward Mode Trace</p>
</div>

Forward mode AD works by, at each elementary operation, calculating both the usual intermediate values (primals) and their derivatives (tangents) simultaneously using standard calculus rules. This mechanism not only yields derivatives but can build entire Jacobian matrices. For a function $f:R^n→R^m$, we set the input **x=a** and initialize its tangent $\dot{x}$ to the unit vector $e_i$ for $i=1,…,n$. Running the function in forward mode then produces the partial derivatives of every output $y_j$ with respect to that single input $x_i$. Each forward pass therefore generates one column of the Jacobian, giving all $\frac{∂y_j}{∂x_i }$.

Jacobian Matrix:

$$
J =
\begin{pmatrix}
\displaystyle \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \displaystyle \frac{\partial y_{1}}{\partial x_{n}} \\
\vdots & \ddots & \vdots \\
\displaystyle \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \displaystyle \frac{\partial y_{m}}{\partial x_{n}}
\end{pmatrix}
$$

Since $f∶R^n→R^m$ has $n$ inputs and each forward mode pass produces one column of the Jacobian, assembling the full $m×n$ Jacobian requires $O(n)$ evaluations. Remember that the Jacobian collects every partial derivative of each output with respect to every input—the very gradients used in optimization.

Forward mode AD naturally extends to compute the Jacobian–vector product (JVP). Given J ϵ $R^{m×n}$ and a direction **r** ϵ $R^n$, the JVP **J∙r** is an $m$-dimensional vector describing how the outputs shift when the inputs are perturbed along **r**.

The power of forward mode AD lies in the fact that you never have to form the entire Jacobian. By selecting an input point and a single perturbation vector **r** , one forward mode evaluation directly yields the JVP **J∙r** with no need to compute all Jacobian columns.


Jacobian-vector Product:

$$
J\,\cdot\mathbf{r}
=
\begin{pmatrix}
\displaystyle \frac{\partial y_1}{\partial x_1} & \cdots & \displaystyle \frac{\partial y_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\displaystyle \frac{\partial y_m}{\partial x_1} & \cdots & \displaystyle \frac{\partial y_m}{\partial x_n}
\end{pmatrix}
\;\cdot\;
\begin{pmatrix}
r_1\\
\vdots\\
r_n
\end{pmatrix}
$$

In summary, forward mode AD is especially efficient for functions $f∶R^n→R^m$ when the number of inputs $n$ is much smaller than the number of outputs $m$. In that scenario, one forward pass suffices to compute the entire Jacobian. Conversely, for a function from $R^n$ to $R$, you need $n$ forward passes to obtain its gradient—making forward mode less practical when $n$ is large.

This distinction matters in neural networks, where the model’s parameters live in $R^n$ but the loss is a single scalar in $R$. Using forward mode AD for gradient based training would be inefficient because $n$ is typically enormous.

Although forward mode avoids numerical pitfalls like instability and expression swell (unlike finite differences or symbolic differentiation), it does not scale well to high dimensional inputs. That limitation is exactly why we turn to AD’s other variant, reverse mode AD for deep learning.


### **Reverse Mode**

At this stage, we introduce reverse mode AD, which mirrors forward mode in spirit but differs in technique. We start by defining adjoints $\bar{v_i}$, each of which represents the partial derivative of a specific output $y_j$, with respect to an intermediate variable $v_i$. Formally, for a function $f∶R^n→R^m$ and indices $i=1,…,n$ and $j=1,…,m$, we set up these adjoints as follows:

$$ \bar{v_i} =  \frac{∂y_j}{∂v_j}$$

In reverse mode AD, we first execute a forward pass exactly as in ordinary code, computing all intermediate values. Unlike forward mode, we don’t calculate derivatives (adjoints) on the fly—instead, we record every operation and its inputs in a computational graph. Once the forward pass is complete, we traverse that graph backward. Using cached intermediate values and the known derivatives of each elementary operation, we apply the chain rule in reverse order to compute each adjoint $\bar{v_i}$. This backward sweep—from final outputs back to inputs—is what gives reverse mode its name.

With intuition behind reverse mode AD, let's take a look at the reverse mode evaluation trace of eq.5 using the same values for the input variables from the Forward Mode Trace.

<div align="center">

| Forward Primal Trace          | Output           | Reverse Adjoint Trace                                                     | Output                |
|-------------------------------|------------------|---------------------------------------------------------------------------|-----------------------|
| $v_{-1} = x_{1}$              | $3$              | $\bar v_{-1} = \bar x_{1} = \bar v_{2}\times\frac{1}{v_{-1}} + \bar v_{1}\times v_{0}$ | $-1 \times (1/3) + 1 \times (-4) = -4.33$ |
| $v_{0} = x_{2}$               | $-4$             | $\bar v_{0} = \bar x_{2} = \bar v_{3}\times 1 + \bar v_{1}\times v_{-1}$         | $1 \times 1 + 1 \times 3 = 4$            |
| $v_{1} = v_{-1} \times v_{0}$ | $3 \times (-4) = -12$ | $\bar v_{1} = \bar v_{3}\times 1$                                         | $1 \times 1 = 1$                       |
| $v_{2} = \ln(v_{-1})$         | $\ln(3) = 1.10$  | $\bar v_{2} = \bar v_{4}\times -1$                                        | $1 \times -1 = -1$                     |
| $v_{3} = v_{1} + v_{0}$       | $-12 + -4 = -16$ | $\bar v_{3} = \bar v_{1} + \bar v_{4}\times 1$                              | $1 \times 1 = 1$                       |
| $v_{4} = v_{2} - v_{3}$       | $-16 - 1.10 = 17.10$ | $\bar v_{4} = \bar y$                                                     | $1$                                    |
| $y = v_{4}$                   | $-17.10$         | $\bar y = \bar v_{4}$                                                       | $1$                                    |                                                                                          | 1                             |
<p style="text-align:center;">Table 3: Reverse Mode Trace</p>
</div>

We begin the backward pass in reverse mode AD by seeding the output’s adjoint $\bar{y}=\frac{∂y}{∂y}=1$. From there, we propagate derivatives back to every variable that influenced $y$, using each operation’s local derivative. Eventually, every input $x$ that contributed to $y$ acquires its adjoint $\bar{x}$.

You may wonder why intermediates like $\bar{v}_{-1}$ and $\bar{v}_{0}$ each receive two contributions when their primal values feed into multiple downstream operations (for example, both $v_2$ and $v_1$ depend on them). Rather than discarding one path, we sum both partial derivatives—so each variable’s adjoint accumulates its total effect on the final output.

Just as forward mode can compute Jacobians by generating one column per pass, reverse mode can generate rows of the Jacobian. If $f∶R^n→R^m$, you set **x=a** and run one reverse mode pass with $\bar{y}=e_j$ for each output index $j$($j=1,…,m$). That pass produces $\frac{∂y_j}{∂x_i }$ for $i=1,…,n$, i.e. one complete row of **J**. Repeating this for all m outputs costs $O(m)$ evaluations.

Moreover, reverse mode naturally yields the vector–Jacobian product (VJP). Given a cotangent (row) vector $r^T$  ϵ $R^{1×m}$, a single backward pass computes $r^T$∙**J**( **J** ϵ $R^{m×n}$), an $1×n$ vector of weighted partials—describing how a perturbation in the outputs propagates back to the inputs.

Vector-Jacobian Product:

$$
\mathbf{r}^T \cdot J
=
\begin{matrix}
(r_1 & \cdots & r_n)^T
\end{matrix}
\cdot
\begin{pmatrix}
\displaystyle \frac{\partial y_1}{\partial x_1} & \cdots & \displaystyle \frac{\partial y_1}{\partial x_n} \\
\vdots & \ddots & \vdots \\
\displaystyle \frac{\partial y_m}{\partial x_1} & \cdots & \displaystyle \frac{\partial y_m}{\partial x_n}
\end{pmatrix}
$$

Vector-Jacobian Product (Alt. Form):

$$
J^T \,.\mathbf{r}
=
\begin{pmatrix}
\displaystyle \frac{\partial y_1}{\partial x_1} & \cdots & \displaystyle \frac{\partial y_m}{\partial x_1} \\
\vdots & \ddots & \vdots \\
\displaystyle \frac{\partial y_1}{\partial x_n} & \cdots & \displaystyle \frac{\partial y_m}{\partial x_n}
\end{pmatrix}^T
\cdot
\begin{pmatrix}
r_1 & \cdots & r_n
\end{pmatrix}
$$

Vector–Jacobian products (VJPs) are central to neural network optimization. We view the Jacobian **J** as the matrix of partial derivatives of a model’s outputs with respect to its inputs, and we interpret $r^T$ as the gradient of a loss function with respect to those outputs. Computing $r^T$∙**J** in this setting directly yields the gradients of the loss with respect to every input parameter—exactly what we need for training—without ever forming the full Jacobian.

In reverse mode AD, one backward pass produces the gradient of a single output with respect to all inputs. To obtain gradients for $m$ outputs, you would run $m$ such passes. Since neural networks typically have $n≫m$ parameters but only one scalar loss, reverse mode is ideal: a single reverse pass computes all $n$ gradients in one shot. Although it uses more memory to store intermediate values, that cost is a small price to pay for avoiding an $O(n)$ explosion in computation time. Ultimately, reverse mode AD is the most efficient choice for gradient based learning, delivering all required derivatives in just one backward sweep.



</div>

## Backpropagation Algorithm
## Gradient-Based Optimization
## give an example and describe the autodiff step-by-step workflow
## Summary. Compare betwwen each type advancement of each types

# Visualization

Code, implement AutoDiff, explain with an example. show backward and forward
Ex: f(x1, x2) = ln(x1) + x1x2 - sin(x2)

https://d2l.ai/chapter_preliminaries/autograd.html

https://homepages.inf.ed.ac.uk/htang2/mlg2022/tutorial-3.pdf


# Exercises (1-4)

1. Why is the second derivative much more expensive to compute than the first derivative?
1. After running the function for backpropagation, immediately run it again and see what happens. Investigate.
1. In the control flow example where we calculate the derivative of `d` with respect to `a`, what would happen if we changed the variable `a` to a random vector or a matrix? At this point, the result of the calculation `f(a)` is no longer a scalar. What happens to the result? How do we analyze this?
1. Let $f(x) = \sin(x)$. Plot the graph of $f$ and of its derivative $f'$. Do not exploit the fact that $f'(x) = \cos(x)$ but rather use automatic differentiation to get the result.



# Exercises (5-8)
1. Let $f(x) = ((\log x^2) \cdot \sin x) + x^{-1}$. Write out a dependency graph tracing results from $x$ to $f(x)$.
1. Use the chain rule to compute the derivative $\frac{df}{dx}$ of the aforementioned function, placing each term on the dependency graph that you constructed previously.
1. Given the graph and the intermediate derivative results, you have a number of options when computing the gradient. Evaluate the result once starting from $x$ to $f$ and once from $f$ tracing back to $x$. The path from $x$ to $f$ is commonly known as *forward differentiation*, whereas the path from $f$ to $x$ is known as backward differentiation.
1. When might you want to use forward, and when backward, differentiation? Hint: consider the amount of intermediate data needed, the ability to parallelize steps, and the size of matrices and vectors involved.

***Reference: origin notebook of d2l as below***

# Automatic Differentiation
:label:`sec_autograd`

Recall from :numref:`sec_calculus`
that calculating derivatives is the crucial step
in all the optimization algorithms
that we will use to train deep networks.
While the calculations are straightforward,
working them out by hand can be tedious and error-prone,
and these issues only grow
as our models become more complex.

Fortunately all modern deep learning frameworks
take this work off our plates
by offering *automatic differentiation*
(often shortened to *autograd*).
As we pass data through each successive function,
the framework builds a *computational graph*
that tracks how each value depends on others.
To calculate derivatives,
automatic differentiation
works backwards through this graph
applying the chain rule.
The computational algorithm for applying the chain rule
in this fashion is called *backpropagation*.

While autograd libraries have become
a hot concern over the past decade,
they have a long history.
In fact the earliest references to autograd
date back over half of a century :cite:`Wengert.1964`.
The core ideas behind modern backpropagation
date to a PhD thesis from 1980 :cite:`Speelpenning.1980`
and were further developed in the late 1980s :cite:`Griewank.1989`.
While backpropagation has become the default method
for computing gradients, it is not the only option.
For instance, the Julia programming language employs
forward propagation :cite:`Revels.Lubin.Papamarkou.2016`.
Before exploring methods,
let's first master the autograd package.


In [ ]:
import torch

## A Simple Function

Let's assume that we are interested
in (**differentiating the function
$y = 2\mathbf{x}^{\top}\mathbf{x}$
with respect to the column vector $\mathbf{x}$.**)
To start, we assign `x` an initial value.


In [ ]:
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

[**Before we calculate the gradient
of $y$ with respect to $\mathbf{x}$,
we need a place to store it.**]
In general, we avoid allocating new memory
every time we take a derivative
because deep learning requires
successively computing derivatives
with respect to the same parameters
a great many times,
and we might risk running out of memory.
Note that the gradient of a scalar-valued function
with respect to a vector $\mathbf{x}$
is vector-valued with
the same shape as $\mathbf{x}$.


In [ ]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad  # The gradient is None by default

(**We now calculate our function of `x` and assign the result to `y`.**)


In [ ]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

[**We can now take the gradient of `y`
with respect to `x`**] by calling
its `backward` method.
Next, we can access the gradient
via `x`'s `grad` attribute.


In [ ]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

(**We already know that the gradient of the function $y = 2\mathbf{x}^{\top}\mathbf{x}$
with respect to $\mathbf{x}$ should be $4\mathbf{x}$.**)
We can now verify that the automatic gradient computation
and the expected result are identical.


In [ ]:
x.grad == 4 * x

tensor([True, True, True, True])

[**Now let's calculate
another function of `x`
and take its gradient.**]
Note that PyTorch does not automatically
reset the gradient buffer
when we record a new gradient.
Instead, the new gradient
is added to the already-stored gradient.
This behavior comes in handy
when we want to optimize the sum
of multiple objective functions.
To reset the gradient buffer,
we can call `x.grad.zero_()` as follows:


In [ ]:
x.grad.zero_()  # Reset the gradient
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

## Backward for Non-Scalar Variables

When `y` is a vector,
the most natural representation
of the derivative of  `y`
with respect to a vector `x`
is a matrix called the *Jacobian*
that contains the partial derivatives
of each component of `y`
with respect to each component of `x`.
Likewise, for higher-order `y` and `x`,
the result of differentiation could be an even higher-order tensor.

While Jacobians do show up in some
advanced machine learning techniques,
more commonly we want to sum up
the gradients of each component of `y`
with respect to the full vector `x`,
yielding a vector of the same shape as `x`.
For example, we often have a vector
representing the value of our loss function
calculated separately for each example among
a *batch* of training examples.
Here, we just want to (**sum up the gradients
computed individually for each example**).


Because deep learning frameworks vary
in how they interpret gradients of
non-scalar tensors,
PyTorch takes some steps to avoid confusion.
Invoking `backward` on a non-scalar elicits an error
unless we tell PyTorch how to reduce the object to a scalar.
More formally, we need to provide some vector $\mathbf{v}$
such that `backward` will compute
$\mathbf{v}^\top \partial_{\mathbf{x}} \mathbf{y}$
rather than $\partial_{\mathbf{x}} \mathbf{y}$.
This next part may be confusing,
but for reasons that will become clear later,
this argument (representing $\mathbf{v}$) is named `gradient`.
For a more detailed description, see Yang Zhang's
[Medium post](https://zhang-yang.medium.com/the-gradient-argument-in-pytorchs-backward-function-explained-by-examples-68f266950c29).


In [ ]:
x.grad.zero_()
y = x * x
y.backward(gradient=torch.ones(len(y)))  # Faster: y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

## Detaching Computation

Sometimes, we wish to [**move some calculations
outside of the recorded computational graph.**]
For example, say that we use the input
to create some auxiliary intermediate terms
for which we do not want to compute a gradient.
In this case, we need to *detach*
the respective computational graph
from the final result.
The following toy example makes this clearer:
suppose we have `z = x * y` and `y = x * x`
but we want to focus on the *direct* influence of `x` on `z`
rather than the influence conveyed via `y`.
In this case, we can create a new variable `u`
that takes the same value as `y`
but whose *provenance* (how it was created)
has been wiped out.
Thus `u` has no ancestors in the graph
and gradients do not flow through `u` to `x`.
For example, taking the gradient of `z = x * u`
will yield the result `u`,
(not `3 * x * x` as you might have
expected since `z = x * x * x`).


In [ ]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

Note that while this procedure
detaches `y`'s ancestors
from the graph leading to `z`,
the computational graph leading to `y`
persists and thus we can calculate
the gradient of `y` with respect to `x`.


In [ ]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Gradients and Python Control Flow

So far we reviewed cases where the path from input to output
was well defined via a function such as `z = x * x * x`.
Programming offers us a lot more freedom in how we compute results.
For instance, we can make them depend on auxiliary variables
or condition choices on intermediate results.
One benefit of using automatic differentiation
is that [**even if**] building the computational graph of
(**a function required passing through a maze of Python control flow**)
(e.g., conditionals, loops, and arbitrary function calls),
(**we can still calculate the gradient of the resulting variable.**)
To illustrate this, consider the following code snippet where
the number of iterations of the `while` loop
and the evaluation of the `if` statement
both depend on the value of the input `a`.


In [ ]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

Below, we call this function, passing in a random value, as input.
Since the input is a random variable,
we do not know what form
the computational graph will take.
However, whenever we execute `f(a)`
on a specific input, we realize
a specific computational graph
and can subsequently run `backward`.


In [ ]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

Even though our function `f` is, for demonstration purposes, a bit contrived,
its dependence on the input is quite simple:
it is a *linear* function of `a`
with piecewise defined scale.
As such, `f(a) / a` is a vector of constant entries
and, moreover, `f(a) / a` needs to match
the gradient of `f(a)` with respect to `a`.


In [ ]:
a.grad == d / a

tensor(True)

Dynamic control flow is very common in deep learning.
For instance, when processing text, the computational graph
depends on the length of the input.
In these cases, automatic differentiation
becomes vital for statistical modeling
since it is impossible to compute the gradient *a priori*.

## Discussion

You have now gotten a taste of the power of automatic differentiation.
The development of libraries for calculating derivatives
both automatically and efficiently
has been a massive productivity booster
for deep learning practitioners,
liberating them so they can focus on less menial.
Moreover, autograd lets us design massive models
for which pen and paper gradient computations
would be prohibitively time consuming.
Interestingly, while we use autograd to *optimize* models
(in a statistical sense)
the *optimization* of autograd libraries themselves
(in a computational sense)
is a rich subject
of vital interest to framework designers.
Here, tools from compilers and graph manipulation
are leveraged to compute results
in the most expedient and memory-efficient manner.

For now, try to remember these basics: (i) attach gradients to those variables with respect to which we desire derivatives; (ii) record the computation of the target value; (iii) execute the backpropagation function; and  (iv) access the resulting gradient.


## Exercises

1. Why is the second derivative much more expensive to compute than the first derivative?
1. After running the function for backpropagation, immediately run it again and see what happens. Investigate.
1. In the control flow example where we calculate the derivative of `d` with respect to `a`, what would happen if we changed the variable `a` to a random vector or a matrix? At this point, the result of the calculation `f(a)` is no longer a scalar. What happens to the result? How do we analyze this?
1. Let $f(x) = \sin(x)$. Plot the graph of $f$ and of its derivative $f'$. Do not exploit the fact that $f'(x) = \cos(x)$ but rather use automatic differentiation to get the result.
1. Let $f(x) = ((\log x^2) \cdot \sin x) + x^{-1}$. Write out a dependency graph tracing results from $x$ to $f(x)$.
1. Use the chain rule to compute the derivative $\frac{df}{dx}$ of the aforementioned function, placing each term on the dependency graph that you constructed previously.
1. Given the graph and the intermediate derivative results, you have a number of options when computing the gradient. Evaluate the result once starting from $x$ to $f$ and once from $f$ tracing back to $x$. The path from $x$ to $f$ is commonly known as *forward differentiation*, whereas the path from $f$ to $x$ is known as backward differentiation.
1. When might you want to use forward, and when backward, differentiation? Hint: consider the amount of intermediate data needed, the ability to parallelize steps, and the size of matrices and vectors involved.


[Discussions](https://discuss.d2l.ai/t/35)
